In [503]:
using Catlab
using Catlab.CategoricalAlgebra
using Catlab.Graphs
using Catlab.Graphs.BasicGraphs
using Catlab.Graphics
using Catlab.Graphics.Graphviz
using Catlab.Graphics.GraphvizGraphs
using Catlab.Graphs.PropertyGraphs
using Catlab.CategoricalAlgebra.CSets
using Catlab.CategoricalAlgebra.FinSets, Catlab.CategoricalAlgebra
using DataStructures
using Colors
using Random

In [504]:
struct Preorder
  carrier:: FinSet
  relation :: SortedSet{Pair{Int,Int}}
  """Construct valid preorder by taking reflexive/transitive closure"""
  function Preorder(carrier:: FinSet, rel :: Vector{Pair{Int,Int}})
    for (a,b) ∈ rel
      a ∈ carrier && b ∈ carrier || error("relation element not in carrier set")
    end
    relation = SortedSet(rel)
    for (a, b, c) in Iterators.product(carrier, carrier, carrier)
      if ((a => b) ∈ relation) && ((b => c) ∈ relation)
        push!(relation, a => c) # enforces relation is transitive
      end
    end
    for i in carrier
      push!(relation, i => i) # enforces that relation is reflexive
    end
    return new(carrier, relation)
  end
end

In [505]:
# function to parse major requirement files
function load_major(path::String, taken_classes::Set{String}=Set{String}())

    file = open(path, "r")

    num_to_class = Dict()
    class_to_num = Dict()

    relations = Vector{Pair{Int, Int}}()

    num_classes = 0
    while !eof(file)
        line = readline(file)
        if length(strip(line, ' ')) != 0
            sp = split(line, ":")

            if length(sp) >= 1
                name = strip(sp[1], ' ')
                if !in(name, taken_classes)
                    num_classes+=1
                    num_to_class[num_classes] = string(name)
                    class_to_num[string(name)] = num_classes
                end

                if length(sp) >= 2
                    reqs_str = strip(sp[2], ['[', ']', ' '])
                    reqs = split(reqs_str, ",")
                    
                    for req in reqs
                        req = strip(req, ' ')
                        if length(req) > 0
                            if !in(req, taken_classes) && in(name, taken_classes)
                                throw("At least one class taken has missing prereqs!")
                            end
                            if !in(req, taken_classes)
                                push!(relations, class_to_num[req] => class_to_num[name])
                            end
                        end
                    end
                end
            end
        end
    end

    close(file)


    return (Preorder(FinSet(num_classes),relations), num_to_class)
end

load_major (generic function with 2 methods)

In [506]:
# load full degree requirements into preorder
cs, cs_mappings = load_major("reqs/CS.reqs")

(Preorder(FinSet(35), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 4, 1 => 5, 1 => 6, 1 => 7, 1 => 9, 1 => 10, 1 => 11  …  28 => 29, 28 => 30, 29 => 29, 29 => 30, 30 => 30, 31 => 31, 32 => 32, 33 => 33, 34 => 34, 35 => 35],
Base.Order.ForwardOrdering())), Dict{Any, Any}(5 => "STA3032", 16 => "COP4600", 20 => "CIS4930-1", 35 => "HumOrSocOrPhy1", 12 => "COP3530", 24 => "CIS4930-5", 28 => "Interdisciplinary-3", 8 => "ENC3246", 17 => "COP4020", 30 => "Interdisciplinary-5"…))

In [507]:
# load degree requirements without some classes that were already taken into preorder
taken_classes = Set{String}(["MAC2311", "COP3502"])
new_cs, mapping = load_major("reqs/CS.reqs", taken_classes)

(Preorder(FinSet(33), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 6, 1 => 10, 1 => 12, 1 => 14, 1 => 15, 1 => 16, 1 => 17  …  26 => 27, 26 => 28, 27 => 27, 27 => 28, 28 => 28, 29 => 29, 30 => 30, 31 => 31, 32 => 32, 33 => 33],
Base.Order.ForwardOrdering())), Dict{Any, Any}(5 => "PHY2048/PHY2048L", 16 => "COP4533", 20 => "CIS4930-3", 12 => "CEN3031", 24 => "Interdisciplinary-1", 28 => "Interdisciplinary-5", 8 => "COP3503", 17 => "CNT4007", 30 => "Humanities2", 1 => "MAC2312"…))

In [508]:
# try to load degree requirements with invalid classes taken
taken_classes = Set{String}(["COP4600"])
new_cs, mapping = load_major("reqs/CS.reqs", taken_classes)

String: "At least one class taken has missing prereqs!"

In [509]:
function top_sort(preorder::Preorder, seed::Int = 0)
    vertices = preorder.carrier
    edges = preorder.relation
    Random.seed!(seed)

    l = []
    q = Queue{String}()
    d = Dict{String, Int}()

    for v in vertices
        d[v] = 0
        for (a,b) in edges
            if v == b && a != b
                d[v]+=1
            end
        end
    end


    
    # randomizes the starting order of the queue
    for v in shuffle!(collect(vertices))
        if d[v] == 0
            enqueue!(q, v)
        end
    end


    while !isempty(q)
        v = dequeue!(q)
        push!(l, v)

        for (a,b) in shuffle!(collect(edges))
            if v == a && a != b
                d[b]-=1
                if d[b] == 0
                    enqueue!(q, b)
                end
            end
        end
    end
    
    return l
end

top_sort (generic function with 2 methods)

In [510]:
# finds elements with no edges in or out
function find_one_offs(preorder::Preorder)
    one_offs = Vector()
    d = Dict()
    for class in collect(preorder.carrier)
        d[class] = true
    end
    for (a,b) in preorder.relation
        if a != b
            d[a] = false
            d[b] = false
        end
    end
    for (class, is_one_off) in d
        if is_one_off
            push!(one_offs, class)
        end
    end
    return Set(one_offs)
end

function generate_schedule(preorder::Preorder, num_semesters::Int=8, seed::Int=0)
    Random.seed!(seed)

    classes = collect(preorder.carrier)
    reqs = preorder.relation

    full_schedule = Vector()

    one_off_classes = find_one_offs(preorder)


    # build dictionary of in counts like in topological sort
    in_counts = Dict()
    for class in classes
        # ignore one off classes for now
        if !(class in one_off_classes)
            in_counts[class] = 0
            for (a,b) in preorder.relation
                if class == b && a != b
                    in_counts[class]+=1
                end
            end
        end
    end


    # schedule classes with prereqs
    for index in 1:num_semesters
        semester_schedule = Vector()

        available_classes = Set(collect(in_counts))

        # find bottoms
        # marking them for deletion
        for (class, in_count) in shuffle!(collect(available_classes))
            if in_count == 0 && length(semester_schedule) < ceil(length(preorder.carrier)/num_semesters)
                push!(semester_schedule, class)
            end
        end

        # delete taken classes
        for class in semester_schedule
            for (a,b) in preorder.relation
                if class == a
                    in_counts[b]-=1
                end
            end
            delete!(in_counts, class)
        end

        push!(full_schedule, semester_schedule)
    end


    # schedule classes with no prereqs
    # prioritizing even credit distribution
    while length(one_off_classes) > 0
        
        # find semester with min number of credits
        min_index = 1
        for (index, semester) in enumerate(full_schedule)
            if length(full_schedule[index]) < length(full_schedule[min_index])
                min_index = index
            end 
        end
        
        class_to_add = collect(one_off_classes)[1]
        push!(full_schedule[min_index], class_to_add)
        delete!(one_off_classes, class_to_add)
    end

    # check that all classes were fit into the schedule
    if length(one_off_classes) > 0 || length(collect(in_counts)) > 0
        throw("Cannot generate schedule!")
    end

    return full_schedule
end

generate_schedule (generic function with 3 methods)

In [511]:
function convert_to_readable(schedule, mapping)
    for (i,semester_schedule) in enumerate(schedule)
        for (j,semester_schedule) in enumerate(semester_schedule)
            schedule[i][j] = mapping[schedule[i][j]]
        end
    end
    return schedule
end

convert_to_readable (generic function with 1 method)

In [512]:
convert_to_readable(generate_schedule(cs, 8), cs_mappings)

8-element Vector{Any}:
 Any["Interdisciplinary-1", "MAC2311", "Social/Behavioral2", "Social/Behavioral1"]
 Any["MAC2312", "PHY2048/PHY2048L", "Interdisciplinary-2", "STA3032", "COP3502"]
 Any["Interdisciplinary-3", "MAC2313", "MAS3114", "COP3503"]
 Any["PHY2049/PHY2049L", "COT3100", "Interdisciplinary-4", "EGS4034"]
 Any["CDA3101", "COP3530", "Interdisciplinary-5", "Humanities1"]
 Any["COP4020", "CIS4930-2", "CIS4930-4", "COP4600", "CIS4301"]
 Any["CIS4930-5", "CIS4930-3", "CIS4930-1", "COP4533", "CNT4007"]
 Any["CEN3031", "Humanities2", "HumOrSocOrPhy1", "ENC3246"]

In [513]:
convert_to_readable(generate_schedule(cs, 10), cs_mappings)

10-element Vector{Any}:
 Any["Interdisciplinary-1", "MAC2311", "HumOrSocOrPhy1", "ENC3246"]
 Any["MAC2312", "PHY2048/PHY2048L", "Interdisciplinary-2", "STA3032"]
 Any["MAC2313", "MAS3114", "Interdisciplinary-3", "COP3502"]
 Any["PHY2049/PHY2049L", "COP3503", "Interdisciplinary-4"]
 Any["Interdisciplinary-5", "COT3100", "Social/Behavioral1"]
 Any["CDA3101", "COP3530", "EGS4034"]
 Any["COP4020", "COP4533", "CEN3031", "CIS4930-4"]
 Any["CIS4301", "COP4600", "CIS4930-1", "CIS4930-2"]
 Any["CIS4930-3", "CNT4007", "CIS4930-5"]
 Any["Humanities2", "Social/Behavioral2", "Humanities1"]

In [514]:
convert_to_readable(generate_schedule(cs, 12), cs_mappings)

12-element Vector{Any}:
 Any["Interdisciplinary-1", "MAC2311", "EGS4034"]
 Any["MAC2312", "PHY2048/PHY2048L", "Interdisciplinary-2"]
 Any["MAS3114", "Interdisciplinary-3", "COP3502"]
 Any["STA3032", "COP3503", "MAC2313"]
 Any["PHY2049/PHY2049L", "Interdisciplinary-4", "COT3100"]
 Any["CDA3101", "Interdisciplinary-5", "COP3530"]
 Any["CEN3031", "CIS4301", "CIS4930-3"]
 Any["CIS4930-1", "CIS4930-4", "CIS4930-5"]
 Any["COP4600", "CIS4930-2", "COP4533"]
 Any["CNT4007", "COP4020", "Humanities1"]
 Any["Humanities2", "HumOrSocOrPhy1", "ENC3246"]
 Any["Social/Behavioral2", "Social/Behavioral1"]

In [515]:
function output_plan(schedule, file_path::String, mapping)
    file = open(file_path, "w")
    for semester_schedule in schedule
        for (index, class) in enumerate(semester_schedule)
            if index == length(semester_schedule)
                print(file, mapping[class])
            else
                print(file, mapping[class] * ", ")
            end
        end
        println(file, "")
    end
    close(file)
end

output_plan (generic function with 2 methods)

In [516]:
# output basic plan
cs, cs_mapping = load_major("reqs/CS.reqs")
schedule = generate_schedule(cs, 8)
output_plan(schedule, "semester_plans/example1.plan", cs_mapping)

In [517]:
# output plan considering the taken classes
taken_classes = Set{String}(["MAC2311", "COP3502"])
new_cs, new_mapping = load_major("reqs/CS.reqs", taken_classes)
new_schedule = generate_schedule(new_cs, 8)
output_plan(new_schedule, "semester_plans/example2.plan", new_mapping)

In [518]:
struct Monotone_map
    domain::Preorder
    codomain::Preorder
    mapping::FinFunction
    function Monotone_map(domain::Preorder, cod::Preorder, mapping::FinFunction)
      ((dom(mapping) == domain.carrier) && (codom(mapping) == cod.carrier)
      ) || error("mapping mismatch")
      return new(domain,cod,mapping)
    end
  end


function is_monotone(mm::Monotone_map)::Bool
    for comp in mm.domain.relation
        mapL = mm.mapping(comp.first)
        mapR = mm.mapping(comp.second)
        mono=false
        
        if(comp.first == comp.second) #reflexive relation
          continue
        end
        if(mapL == mapR) #pre req and class taken in same semeester
          return false;
        end

        for comparison in mm.codomain.relation

            if(mapL == comparison.first && mapR == comparison.second)
                mono=true
            end
        end
        if(!mono)
            return false
        end
    end
    return true
end

is_monotone (generic function with 1 method)

In [519]:
#check that the generated schedule creates a monotone map onto 8 semesters
cs, cs_mapping = load_major("reqs/CS.reqs")
schedule = generate_schedule(cs, 8)

semester_preorder = Preorder(FinSet(8), [1=>2, 2=>3, 3=>4, 4=>5, 5=>6, 6=>7, 7=>8])

indices = Vector{Int}(undef,length(cs.carrier))
for (index,semester) in enumerate(schedule)
    for class in semester
        indices[class] = index
    end
end




f = FinFunction(indices, cs.carrier, semester_preorder.carrier)
schedule_map = Monotone_map(cs, semester_preorder, f)

Monotone_map(Preorder(FinSet(35), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 4, 1 => 5, 1 => 6, 1 => 7, 1 => 9, 1 => 10, 1 => 11  …  28 => 29, 28 => 30, 29 => 29, 29 => 30, 30 => 30, 31 => 31, 32 => 32, 33 => 33, 34 => 34, 35 => 35],
Base.Order.ForwardOrdering())), Preorder(FinSet(8), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 4, 1 => 5, 1 => 6, 1 => 7, 1 => 8, 2 => 2, 2 => 3  …  5 => 5, 5 => 6, 5 => 7, 5 => 8, 6 => 6, 6 => 7, 6 => 8, 7 => 7, 7 => 8, 8 => 8],
Base.Order.ForwardOrdering())), FinFunction([1, 2, 3, 3, 2, 2, 4, 8, 2, 3, 4, 5, 5, 8, 6, 6, 6, 7, 7, 7, 6, 7, 6, 7, 4, 1, 2, 3, 4, 5, 5, 8, 1, 1, 8], 35, 8))

In [520]:
# check that the generated schedule specifies a monotone map between the req preorder and the semester preorder
is_monotone(schedule_map)

In [ ]:
courses = Preorder(FinSet(4), [1=>2, 2=>3, 2=>4])
semesters = Preorder(FinSet(3), [1=>2, 2=>3])

is_monotone(Monotone_map(courses, semesters, FinFunction([1,2,3,3], FinSet(4), FinSet(3))))

In [ ]:
is_monotone(Monotone_map(courses, semesters, FinFunction([2,1,3,3], FinSet(4), FinSet(3))))

In [ ]:
is_monotone(Monotone_map(courses, semesters, FinFunction([1,1,2,3], FinSet(4), FinSet(3))))

In [ ]:
taken_classes = Set{String}(["MAC2311", "COP3502"])
new_cs, mapping = load_major("reqs/CS.reqs", taken_classes)
